In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os
import gc
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input
from tensorflow.keras.models import Model, Sequential, load_model

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def decay(epochs):
    init = 1e-3
    drop = 10
    ratio = 0.9
    return max(5e-5, (init * (ratio ** (epochs//drop))))

es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
lrs = callbacks.LearningRateScheduler(decay, verbose=0)


In [3]:
class GAN(keras.Model):
    def __init__(self, x_shape, y_shape):
        super(GAN, self).__init__()
        self.x_shape  = x_shape
        self.y_shape = y_shape
        
        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()
    
    def compile(self, g_optim, d_optim, d_loss_fn, recon_loss_fn):
        super(GAN, self).compile()
        self.g_optim = g_optim
        self.d_optim = d_optim
        self.d_loss_fn = d_loss_fn
        self.recon_loss_fn = recon_loss_fn
        
    def build_generator(self):
        activation = mish
        inputs = Input(shape=self.x_shape)
        
        x = Dense(128, kernel_initializer='he_normal')(inputs)
        x = Activation(activation)(x)
        x = Dense(256, kernel_initializer='he_normal')(x)
        x = Activation(activation)(x)
        x = Dense(512, kernel_initializer='he_normal')(x)
        x = Activation(activation)(x)
        
        outputs = Dense(self.y_shape[0], kernel_initializer='he_normal')(x)
        return Model(inputs, outputs)
    
    def build_discriminator(self):
        inputs1 = Input(shape = self.x_shape) # feature
        inputs2 = Input(shape = self.y_shape) # target
        
        inputs = Concatenate()([inputs1, inputs2])
        
        x = Dense(512)(inputs)
        x = Dense(256)(x)
        x = Dense(128)(x)
        
        outputs = Dense(1, activation='sigmoid')(x)
        
        return Model([inputs1, inputs2], [outputs, inputs2])
    
    def train_step(self, data):
        x, y = data
        batch_size = tf.shape(x)[0]
        
        preds_y = self.generator(x)
        all_y = tf.concat([y, preds_y], 0)
        all_x = tf.concat([x, x], 0)
        
        fake_labels = tf.ones((batch_size, 1))*0
        real_labels = tf.ones((batch_size, 1))
        labels = tf.concat([fake_labels, real_labels], 0)
        
        # keras official tutorial saids add noise to label is important trick
        # labels = 0.05*tf.random.uniform(tf.shape(labels))
        
        # disc / gen alternatively
        with tf.GradientTape() as tape:
            preds_ = self.generator(x)
            validity, _ = self.discriminator([all_x, all_y])
            d_loss = self.d_loss_fn(labels, validity)
            
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optim.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        with tf.GradientTape() as tape:
            preds = self.generator(x)
            validity, preds = self.discriminator([x, preds])
            g_loss = self.recon_loss_fn([real_labels, y], [validity, preds])
            
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optim.apply_gradients(zip(grads, self.generator.trainable_weights))

        return {'d_loss': d_loss, 'g_loss': g_loss}
    
    def sampler(self, batch_size):
        return tf.random.normal(shape=(batch_size, self.z_dim))


In [4]:
gan = GAN((120, ), (3, ))

In [5]:
def recon_loss(y_trues, y_preds):
    t1, t2 = y_trues
    p1, p2 = y_preds
    
    validity_loss = losses.BinaryCrossentropy()(t1, p1)
    recon_loss = losses.MeanAbsoluteError()(t2, p2)
    
    return validity_loss+recon_loss*100

In [6]:
gan.compile(
    optimizers.Adam(2e-4),
    optimizers.Adam(2e-4),
    losses.BinaryCrossentropy(),
    recon_loss
         )

In [7]:
X = np.random.normal(0, 1, (1000, 120))
y = np.random.normal(0, 1, (1000, 3))

In [8]:
gan.fit(X, y,
     epochs=20)

Epoch 1/20
32/32 [==============================] - 0s 6ms/step - d_loss: 0.6071 - g_loss: 1.6666
Epoch 2/20
32/32 [==============================] - 0s 6ms/step - d_loss: 0.4681 - g_loss: 1.4000
Epoch 3/20
32/32 [==============================] - 0s 6ms/step - d_loss: 0.4016 - g_loss: 1.3493
Epoch 4/20
32/32 [==============================] - 0s 6ms/step - d_loss: 0.3577 - g_loss: 1.2895
Epoch 5/20
32/32 [==============================] - 0s 5ms/step - d_loss: 0.3189 - g_loss: 1.2676
Epoch 6/20
32/32 [==============================] - 0s 5ms/step - d_loss: 0.2712 - g_loss: 1.2343
Epoch 7/20
32/32 [==============================] - 0s 5ms/step - d_loss: 0.2609 - g_loss: 1.1943
Epoch 8/20
32/32 [==============================] - 0s 6ms/step - d_loss: 0.2077 - g_loss: 1.1954
Epoch 9/20
32/32 [==============================] - 0s 6ms/step - d_loss: 0.1878 - g_loss: 1.1665
Epoch 10/20
32/32 [==============================] - 0s 5ms/step - d_loss: 0.1675 - g_loss: 1.1305
Epoch 11/20
32/32 [

In [9]:
gan.generator.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120)]             0         
_________________________________________________________________
dense (Dense)                (None, 128)               15488     
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_2 (Activation)    (None, 512)              

In [10]:
gan.discriminator.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 120)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 123)          0           input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 512)          63488       concatenate[0][0]     